<a href="https://colab.research.google.com/github/PedroMurta/desafio-analista/blob/main/Desafio_Analista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar as bibliotecas que serão usadas durante o projeto
import psycopg2 # biblioteca para trabalhar com conexão do banco de dados
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

In [2]:
# Tabela Policies
df_policies = pd.read_excel('https://github.com/PedroMurta/desafio-analista/blob/main/fake_allocation_policies.xlsx?raw=true', names=['class_name', 'conservador', 'moderado-conservador', 'moderado', 'moderado-agressivo', 'agressivo'])

# Tabela Position
df_position = pd.read_csv('https://raw.githubusercontent.com/PedroMurta/desafio-analista/main/fake_position.csv')



In [ ]:
# Configuração da conexão com o banco de dados no Railway
conn = psycopg2.connect(host='containers-us-west-59.railway.app',
                        dbname='railway', user='postgres', 
                        password='sMjvuuZSLqSDOYV84IcF',
                        port="6487")



# Leitura dos dados do desafio
position_data = pd.read_csv('https://raw.githubusercontent.com/PedroMurta/Desafio-Dashboard/main/fake_position.csv')
allocation_policies_data = pd.read_excel('https://github.com/PedroMurta/Desafio-Dashboard/blob/main/fake_allocation_policies.xlsx?raw=true')



# Criação das tabelas com as características dos arquivos lidos 
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE position (
            account_code FLOAT,
            account_suitability TEXT,
            asset_name TEXT,
            asset_cnpj TEXT,
            class_name TEXT,
            position_value FLOAT
            
             
        );
    """)
    cur.execute("""
        CREATE TABLE policy (            
            Classe TEXT,
            Conservador FLOAT,
            "Moderado Conservador" FLOAT,
            Moderado FLOAT,
            "Moderado Agressivo" FLOAT,
            Agressivo FLOAT,
            PRIMARY KEY (Classe)            
        );
    """)


# Inserção dos dados
with conn.cursor() as cur:
    for _, row in position_data.iterrows():
        cur.execute("""
            INSERT INTO position (account_code, account_suitability, asset_name, asset_cnpj, class_name, position_value)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['account_code'], row['account_suitability'], row['asset_name'], row['asset_cnpj'], row['class_name'],  row['position_value']))
    for _, row in allocation_policies_data.iterrows():
        cur.execute("""
            INSERT INTO policy (Classe, Conservador, "Moderado Conservador", Moderado, "Moderado Agressivo", Agressivo)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['Classe'], row['Conservador'], row['Moderado Conservador'], row['Moderado'], row['Moderado Agressivo'], row['Agressivo']))

# Salvar os dados e fazer o commit
conn.commit()


In [ ]:
cur = conn.cursor()
cur.execute("""
WITH
  current_allocation AS (
    SELECT
      account_code,
      class_name,
      SUM(position_value) AS current_value,
      SUM(position_value) / NULLIF((SELECT SUM(position_value) FROM position p2 WHERE p2.account_code = p1.account_code), 0) AS current_allocation
    FROM
      position p1
    GROUP BY
      account_code,
      class_name
  ),
  recommended_allocation AS (
    SELECT
      Classe AS class_name,
      policy.Conservador AS conservador,
      policy."Moderado Conservador" AS mod_conservador,
      policy.Moderado AS moderado,
      policy."Moderado Agressivo" AS mod_agressivo,
      policy.Agressivo AS agressivo
    FROM
      policy
  ),
  distance AS (
    SELECT
      ca.account_code,
      ca.class_name,
      ABS(ca.current_allocation - ra.conservador) AS dist_conservador,
      ABS(ca.current_allocation - ra.mod_conservador) AS dist_mod_conservador,
      ABS(ca.current_allocation - ra.moderado) AS dist_moderado,
      ABS(ca.current_allocation - ra.mod_agressivo) AS dist_mod_agressivo,
      ABS(ca.current_allocation - ra.agressivo) AS dist_agressivo
    FROM
      current_allocation ca
      INNER JOIN recommended_allocation ra ON ca.class_name = ra.class_name
  )
SELECT
  account_code,
  AVG((dist_conservador + dist_mod_conservador + dist_moderado + dist_mod_agressivo + dist_agressivo) / 5) AS adherence
FROM
  distance
GROUP BY
  account_code
ORDER BY
  account_code

""")

results = cur.fetchall()

# Print the results
for row in results:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()

In [ ]:
cur = conn.cursor()

cur.execute("""
  SELECT 
      p.account_code, 
      p.class_name, 
      SQRT(
          POWER(p.position_value - pol.conservador, 2) +
          POWER(p.position_value - pol."Moderado Conservador", 2) +
          POWER(p.position_value - pol.moderado, 2) +
          POWER(p.position_value - pol."Moderado Agressivo", 2) +
          POWER(p.position_value - pol.agressivo, 2)
      ) AS aderencia
  FROM position p
  LEFT JOIN policy pol
  ON p.class_name = pol.Classe""")


# Fetch the results
results = cur.fetchall()

# Print the results
for row in results:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()

In [3]:
policies = df_policies.copy()
position = df_position.copy()

# Análise Exploratória e Tratamento


#### Entendendo a base de dados:

Os dados que serão usados neste projeto são dados fictícios do desafio proposto.

Os dados do arquivo *fake_position.csv* é composto por 6 colunas:

- `account_code`: código da conta do cliente

- `account_suitability`: perfil suitability da conta do cliente

- `asset_name`: nome do ativo financeiro na carteira do cliente
- `asset_cnpj`: CNPJ do ativo financeiro na carteira do cliente
- `class_name`: classe do ativo financeiro na carteira do cliente
- `position_value`: valor da posição do ativo financeiro na carteira do cliente

Os dados do arquivo *fake_allocation_policies.xlsx* também é composto por 6 colunas:

- `Classe`: nome da classe de ativos

- `Conservador`: percentual de alocação para a classe de ativos do perfil Conservador

- `Moderado Conservador`: percentual de alocação para a classe de ativos do perfil Moderado Conservador

- `Moderado`: percentual de alocação para a classe de ativos do perfil  Moderado

- `Moderado Agressivo`: percentual de alocação para a classe de ativos do perfil Moderado Agressivo

- `Agressivo`: percentual de alocação para a classe de ativos do perfil Agressivo

In [4]:
# Criar uma função que retorna a métrica de cada classe que busca os valores das colunas da tabela policies equivalentes a cada linha das colunas account_suitability + nome da classe

def get_metrica(row):
    politica = row['account_suitability']
    if politica == 'conservador':
        return row['conservador']

    elif politica == 'moderado conservador':
        return row['moderado_conservador']

    elif politica == 'moderado':
        return row['moderado']

    elif politica == 'moderado agressivo':
        return row['moderado_agressivo']

    elif politica == 'agressivo':
        return row['agressivo']
    else:
        return None

# Merge das duas tabelas 
merged = pd.merge(position, policies, on='class_name')

# Aplicar a função na tabela agrupada
merged['metrica'] = merged.apply(get_metrica, axis=1)

merged.head()

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value,conservador,moderado-conservador,moderado,moderado-agressivo,agressivo,metrica
0,65547.00,moderado,CDB PRE DU CDB120CAG00,NaN,Renda Fixa Pré-Fixada,133886.22,0.05,0.08,0.10,0.11,0.10,0.10
1,65547.00,moderado,CDB PRE DU CDB320A2QHT,NaN,Renda Fixa Pré-Fixada,61127.17,0.05,0.08,0.10,0.11,0.10,0.10
2,65547.00,moderado,CDB PRE DU CDB420IBE9D,NaN,Renda Fixa Pré-Fixada,206855.53,0.05,0.08,0.10,0.11,0.10,0.10
3,65547.00,moderado,CDB PRE DU CDB321ALNBI,NaN,Renda Fixa Pré-Fixada,47910.69,0.05,0.08,0.10,0.11,0.10,0.10
4,65547.00,moderado,LCI PRE 21K00148900,NaN,Renda Fixa Pré-Fixada,71442.63,0.05,0.08,0.10,0.11,0.10,0.10


In [5]:
# Agrupar os dados por classe pare ter os valores de cada classe de ativos na carteira dos clientes
df_agrupado_classe = position.groupby(['account_code', 'account_suitability','class_name']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_classe.head(7)

,account_code,account_suitability,class_name,position_value
0,2429.00,moderado,Renda Fixa Inflação,134310122.39
1,2429.00,moderado,Renda Fixa Pré-Fixada,296872.54
2,2429.00,moderado,Renda Fixa Pós-Fixada,6425.97
3,2429.00,moderado,Reserva de Liquidez,13061.73
4,2429.00,moderado,Saldo em Conta,0.00
5,6777.00,moderado-conservador,Alternativos,32197.54
6,6777.00,moderado-conservador,Multimercado,20820.84


In [6]:
# Agrupar os dados pelas class_names para buscar a métrica e os valores atuais de cada ativo dos clientes
df_agrupado_classes = merged.groupby(['account_code', 'account_suitability','class_name', 'metrica']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_classes.head(7)

,account_code,account_suitability,class_name,metrica,position_value
0,2429.00,moderado,Renda Fixa Inflação,0.24,134310122.39
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,296872.54
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,6425.97
3,2429.00,moderado,Saldo em Conta,0.00,0.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32
5,20921.00,agressivo,Renda Variável,0.20,4251.29
6,20921.00,agressivo,Saldo em Conta,0.00,3.65


In [7]:
# Agrupar os dados pelo perfil suitability dos clientes para buscar os valores totais que cada cliente tem aplicado
df_agrupado_suitability = df_agrupado_classe.groupby(['account_code', 'account_suitability']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_suitability 

,account_code,account_suitability,position_value
0,2429.00,moderado,134626482.63
1,6777.00,moderado-conservador,470566.94
2,7826.00,moderado-conservador,183962.86
3,18073.00,moderado-conservador,65559838.14
4,20921.00,agressivo,27915.92
5,32263.00,moderado-conservador,278274189.45
6,32449.00,moderado-conservador,1726743.04
7,45848.00,moderado,125103.31
8,46168.00,moderado,162024.01
9,46744.00,moderado-agressivo,37913.63


In [8]:
df_agrupado_suitability['total_value'] = df_agrupado_suitability['position_value']

In [9]:
# Melhorar a visualização da relação do valor total de cada conta
total_agrupado = pd.merge(df_agrupado_suitability, df_agrupado_classe, on='account_code', how='left')
total_agrupado = total_agrupado[['account_code', 'total_value']].drop_duplicates()
total_agrupado.head()

,account_code,total_value
0,2429.00,134626482.63
5,6777.00,470566.94
13,7826.00,183962.86
19,18073.00,65559838.14
27,20921.00,27915.92


In [10]:
# Criar uma função que retorna a métrica de cada classe que busca os valores das colunas da tabela policies equivalentes a cada linha das colunas account_suitability + nome da classe

def get_metrica(row):
    politica = row['account_suitability']
    if politica == 'conservador':
        return row['conservador']

    elif politica == 'moderado conservador':
        return row['moderado_conservador']

    elif politica == 'moderado':
        return row['moderado']

    elif politica == 'moderado agressivo':
        return row['moderado_agressivo']

    elif politica == 'agressivo':
        return row['agressivo']
    else:
        return None

# Merge das duas tabelas 
merged = pd.merge(position, policies, on='class_name')

# Aplicar a função na tabela agrupada
merged['metrica'] = merged.apply(get_metrica, axis=1)

merged.head()

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value,conservador,moderado-conservador,moderado,moderado-agressivo,agressivo,metrica
0,65547.00,moderado,CDB PRE DU CDB120CAG00,NaN,Renda Fixa Pré-Fixada,133886.22,0.05,0.08,0.10,0.11,0.10,0.10
1,65547.00,moderado,CDB PRE DU CDB320A2QHT,NaN,Renda Fixa Pré-Fixada,61127.17,0.05,0.08,0.10,0.11,0.10,0.10
2,65547.00,moderado,CDB PRE DU CDB420IBE9D,NaN,Renda Fixa Pré-Fixada,206855.53,0.05,0.08,0.10,0.11,0.10,0.10
3,65547.00,moderado,CDB PRE DU CDB321ALNBI,NaN,Renda Fixa Pré-Fixada,47910.69,0.05,0.08,0.10,0.11,0.10,0.10
4,65547.00,moderado,LCI PRE 21K00148900,NaN,Renda Fixa Pré-Fixada,71442.63,0.05,0.08,0.10,0.11,0.10,0.10


In [11]:
# Agrupar os dados pelas class_names para buscar a métrica e os valores atuais de cada ativo dos clientes
df_agrupado_classes = merged.groupby(['account_code', 'account_suitability','class_name', 'metrica']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_classes.head(7)

,account_code,account_suitability,class_name,metrica,position_value
0,2429.00,moderado,Renda Fixa Inflação,0.24,134310122.39
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,296872.54
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,6425.97
3,2429.00,moderado,Saldo em Conta,0.00,0.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32
5,20921.00,agressivo,Renda Variável,0.20,4251.29
6,20921.00,agressivo,Saldo em Conta,0.00,3.65


In [12]:
# Criar nova tabela que insere os valores totais de acordo com cada account_code para facilitar cálculos futuros
df = pd.merge(df_agrupado_classes, total_agrupado, on='account_code')
df.head(7)

,account_code,account_suitability,class_name,metrica,position_value,total_value
0,2429.00,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63
3,2429.00,moderado,Saldo em Conta,0.00,0.00,134626482.63
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92
5,20921.00,agressivo,Renda Variável,0.20,4251.29,27915.92
6,20921.00,agressivo,Saldo em Conta,0.00,3.65,27915.92


In [13]:
# Renomear coluna position_value para valor_atual para facilitar a visualização
df['valor_atual'] = df['position_value']
df.drop(columns='position_value', inplace=True)
df.head(7)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual
0,2429.00,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97
3,2429.00,moderado,Saldo em Conta,0.00,134626482.63,0.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32
5,20921.00,agressivo,Renda Variável,0.20,27915.92,4251.29
6,20921.00,agressivo,Saldo em Conta,0.00,27915.92,3.65


In [14]:
# Para encontrar o valor esperado de cada classe dos clientes basta pegar o valor total e multiplicar pela métrica
df['valor_esperado'] = round(df['total_value'] * df['metrica'], 3)
df.head(7)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado
0,2429.00,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79
3,2429.00,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80
5,20921.00,agressivo,Renda Variável,0.20,27915.92,4251.29,5583.18
6,20921.00,agressivo,Saldo em Conta,0.00,27915.92,3.65,0.00


In [15]:
# Para facilitar a visualização novamente, criar uma coluna de valores atuais em porcentagem (%)
df['valor_porcentagem_atual'] = round(df['valor_atual'] / df['total_value'] * 100, 3)
df.head(13)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual
0,2429.00,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01
3,2429.00,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49
5,20921.00,agressivo,Renda Variável,0.20,27915.92,4251.29,5583.18,15.23
6,20921.00,agressivo,Saldo em Conta,0.00,27915.92,3.65,0.00,0.01
7,45848.00,moderado,Multimercado,0.22,125103.31,26015.53,27522.73,20.80
8,45848.00,moderado,Renda Fixa Inflação,0.24,125103.31,5604.31,30024.79,4.48
9,45848.00,moderado,Renda Fixa Pré-Fixada,0.10,125103.31,4161.47,12510.33,3.33


In [16]:
df['distancia_porcentagem'] = ((df['valor_porcentagem_atual'] - (df['metrica'] * 100)) **2) ** 0.5
df.head(7)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia_porcentagem
0,2429.00,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77,75.77
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22,9.78
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01,30.00
3,2429.00,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00,0.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49,67.49
5,20921.00,agressivo,Renda Variável,0.20,27915.92,4251.29,5583.18,15.23,4.77
6,20921.00,agressivo,Saldo em Conta,0.00,27915.92,3.65,0.00,0.01,0.01


In [17]:
# Encontrar o valor da distância para fazer o cálculo da Distância Euclidiana
# A fórmula da distância euclidiana está na imagem acima. elevar o resultado em 0.5 é a mesma coisa que tirar a raiz quadrada
df['distancia_euclidiana'] = ((df['metrica'] - df['distancia_porcentagem'])**2)**0.5
df.head(7)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia_porcentagem,distancia_euclidiana
0,2429.00,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77,75.77,75.53
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22,9.78,9.68
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01,30.00,29.70
3,2429.00,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00,0.00,0.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49,67.49,67.44
5,20921.00,agressivo,Renda Variável,0.20,27915.92,4251.29,5583.18,15.23,4.77,4.57
6,20921.00,agressivo,Saldo em Conta,0.00,27915.92,3.65,0.00,0.01,0.01,0.01


In [18]:
df['aderencia'] = round(1 / (1 + df['distancia_euclidiana']),3)
df.head()

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia_porcentagem,distancia_euclidiana,aderencia
0,2429.00,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77,75.77,75.53,0.01
1,2429.00,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22,9.78,9.68,0.09
2,2429.00,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01,30.00,29.70,0.03
3,2429.00,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00,0.00,0.00,1.00
4,20921.00,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49,67.49,67.44,0.01


In [19]:
# Salvar o arquivo como csv para usar na criação dos dashboards
df.to_csv('database.csv')

In [20]:
df['account_code'] = df['account_code'].apply(lambda x: f'conta_{str(x)}')
df.head()

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia_porcentagem,distancia_euclidiana,aderencia
0,conta_2429.0,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77,75.77,75.53,0.01
1,conta_2429.0,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22,9.78,9.68,0.09
2,conta_2429.0,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01,30.00,29.70,0.03
3,conta_2429.0,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00,0.00,0.00,1.00
4,conta_20921.0,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49,67.49,67.44,0.01


In [24]:
df_peso_carteira = df_position.groupby(['asset_name', 'account_suitability']).agg({'position_value': 'sum'}).reset_index().sort_values('position_value', ascending=False)
df_peso_carteira = df_peso_carteira.iloc[:6]
df_peso_carteira.head()

,asset_name,account_suitability,position_value
216,NTN-B 760199,moderado,999540099.11
218,NTN-B 760199,moderado-conservador,267309486.11
107,CRI 21K0001807,moderado,133943599.00
56,CDB FLU CDB321CGU1J,moderado,112318562.00
126,DEB FLU U LORTA7,moderado,70103916.00


In [25]:
aderencia_conta = df.groupby(['account_code']).agg({'aderencia': 'mean'}).reset_index().sort_values(by='aderencia', ascending=False)
aderencia_conta = aderencia_conta[: 2]
aderencia_conta.head()
# A conta 78059.00 possui apenas 2 registros, por isso o valor discrepante

,account_code,aderencia
10,conta_78059.0,1.00
11,conta_82356.0,0.43


In [28]:
df_position['asset_name'].value_counts()

Saldo em Conta                         25
VLGI Asset DI FIRF CP                  25
NTN-B 760199                           21
Vinland RF Ativo Advisory LP FIC FI    13
VIVT3                                  11
                                       ..
LCI PRE 23A00125741                     1
CDB FLU CDB1230XR9F                     1
LCA PRE 23C01237215                     1
CDB FLU CDB123O1RLA                     1
VBI Crédito - AVBI11                    1
Name: asset_name, Length: 278, dtype: int64

In [30]:
df.head()

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia_porcentagem,distancia_euclidiana,aderencia
0,conta_2429.0,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77,75.77,75.53,0.01
1,conta_2429.0,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22,9.78,9.68,0.09
2,conta_2429.0,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01,30.00,29.70,0.03
3,conta_2429.0,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00,0.00,0.00,1.00
4,conta_20921.0,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49,67.49,67.44,0.01


In [42]:
df_class_maior_aderencia = df_position.groupby(['class_name']).agg({'position_value': 'sum'}).reset_index().sort_values('position_value', ascending=False)
df_class_maior_aderencia.head()

,class_name,position_value
3,Renda Fixa Inflação,1659457982.44
0,Alternativos,115815921.93
4,Renda Fixa Pré-Fixada,74481748.89
5,Renda Fixa Pós-Fixada,22764826.30
7,Reserva de Liquidez,4868852.98


In [33]:
df_position[df_position['account_code'] == 78059.00]

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value
287,78059.00,moderado,VLGI Asset DI FIRF CP,47729760000106.00,Reserva de Liquidez,59031.72
528,78059.00,moderado,Saldo em Conta,NaN,Saldo em Conta,0.00


In [34]:
position.head()

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value
0,65547.00,moderado,CDB PRE DU CDB120CAG00,NaN,Renda Fixa Pré-Fixada,133886.22
1,65547.00,moderado,CDB PRE DU CDB320A2QHT,NaN,Renda Fixa Pré-Fixada,61127.17
2,65547.00,moderado,CDB PRE DU CDB420IBE9D,NaN,Renda Fixa Pré-Fixada,206855.53
3,65547.00,moderado,CDB PRE DU CDB321ALNBI,NaN,Renda Fixa Pré-Fixada,47910.69
4,65547.00,moderado,LCI PRE 21K00148900,NaN,Renda Fixa Pré-Fixada,71442.63


In [35]:
asset_class = merged.groupby(['account_code', 'asset_name', 'account_suitability','class_name', 'metrica']).agg({'position_value': 'sum'}).reset_index()
asset_class.head(7)

,account_code,asset_name,account_suitability,class_name,metrica,position_value
0,2429.00,CDB FLU CDB122DP4IN,moderado,Renda Fixa Inflação,0.24,114034.32
1,2429.00,CDB PRE DU CDB122DP2ZX,moderado,Renda Fixa Pré-Fixada,0.10,148965.51
2,2429.00,CDB PRE DU CDB122E1Y4T,moderado,Renda Fixa Pré-Fixada,0.10,147907.03
3,2429.00,CRA FLU CRA022004MS,moderado,Renda Fixa Inflação,0.24,25263.79
4,2429.00,CRI 21K0001807,moderado,Renda Fixa Inflação,0.24,133943599.00
5,2429.00,CRI 21K0001812,moderado,Renda Fixa Inflação,0.24,134559.38
6,2429.00,NTN-B 760199,moderado,Renda Fixa Inflação,0.24,92665.90


In [46]:
asset_class = asset_class.groupby(['asset_name', 'class_name']).agg({'position_value': 'sum'}).reset_index()
asset_class.head()

,asset_name,class_name,position_value
0,ARX Elbrus Advisory FIC INFRA RF,Renda Fixa Inflação,16038.00
1,ASAI3,Renda Variável,385.50
2,Absolute Hedge FIC FIM,Multimercado,4536.25
3,Absolute Vertex Advisory FIC FIM,Multimercado,79927.81
4,BNP Índice Fundos Internacionais Morningstar -...,Alternativos,37000.00


In [48]:
asset_class_a = asset_class.groupby(['asset_name']).agg({'position_value': 'sum'}).reset_index().sort_values('position_value', ascending=False)
asset_class_a.head()

,asset_name,position_value
100,NTN-B 760199,999557094.30
53,CRI 21K0001807,133943599.00
24,CDB FLU CDB321CGU1J,112318562.00
62,DEB FLU U LORTA7,70103916.00
134,XP Man Fundo Multimercado - IPCA + Alta Ilimit...,56791126.00
